# Compute Analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial import KDTree

In [ ]:
# Read the file in
df = pd.read_parquet('B19001_no_geometry.parquet')

In [ ]:
# Build the KD Tree
centroids = df[['CentroidX', 'CentroidY', 'CentroidZ']].to_numpy()
c_tree = KDTree(centroids)

Look at https://www.bls.gov/cex/tables.htm 
https://www.bls.gov/cex/2019/combined/income.xlsx

Household ops, housekeeping supplies, laundry supplies, postage, textiles, furniture, floor coverings, small appliances, misc equipment, clothing, av equipment, pets/toys/hobbies/playground equipment, personal care, reading

| <15          | 15-30        | 30-40        | 40-50        | 50-70        | 70-100       | 100-150      | 150-200      | >200         |
|--------------|--------------|--------------|--------------|--------------|--------------|--------------|--------------|--------------|
|  $ 4,370.00  |  $ 5,767.00  |  $ 6,681.00  |  $ 7,959.00  |  $ 8,712.00  |  $10,655.00  |  $13,396.00  |  $17,638.00  |  $26,174.00  |

In [ ]:
# Organize the data into expected spending per income group.
levels = [
    4370,
    4370,
    5767,
    5767,
    5767,
    6681,
    6681,
    7959,
    7959,
    8712,
    8712,
    10655,
    13396,
    13396,
    17638,
    26174
]
level_columns = df.columns[13:29]

In [ ]:
# Compute the expected amount available to spend per entire block group for all block groups in the US
spending_potential = np.einsum('ij,j->i', df[level_columns], levels)
spending_potential[np.where(np.isnan(spending_potential))] = 0
spending_potential[spending_potential == np.inf] = 0
grounded_potential = spending_potential.copy()
grounded_potential[np.abs(grounded_potential - grounded_potential.mean()) > 4 * grounded_potential.std()] = grounded_potential.mean()

In [ ]:
# Pick the top num_search block groups with highest spending potential, looking min_search_dist away for the subsequent best block groups.
num_search = 40
min_search_dist = 50

available_indexes = np.arange(len(grounded_potential))
top_locations = np.zeros(num_search, dtype=np.int)

for idx in range(num_search):
    best_index = available_indexes[grounded_potential[available_indexes].argmax()]
    top_locations[idx] = best_index
    
    off_limits = c_tree.query_ball_point(centroids[best_index], min_search_dist)
    available_indexes = available_indexes[~np.isin(available_indexes, off_limits)]

In [ ]:
# Get all info for the best locations.
best_locs = df.iloc[top_locations]

In [ ]:
# Save the result.
best_locs.to_csv('SearchResult.csv')

In [ ]:
# View the names of the top selected sites
best_locs['NAME'].to_list()

In [ ]:
# Save the top 20.

neighboring_areas = [np.array(x) for x in c_tree.query_ball_point(centroids[top_locations], 20)]
region_arr = np.zeros((len(neighboring_areas), max([len(x) for x in neighboring_areas])), dtype=np.int)

for i in range(len(neighboring_areas)):
    region_arr[i, :len(neighboring_areas[i])] = neighboring_areas[i]
    
np.savetxt('TargetRegions.csv', region_arr, delimiter=',')

In [ ]:
# Save the top 10.

neighboring_areas = [np.array(x) for x in c_tree.query_ball_point(centroids[top_locations], 10)]
region_arr = np.zeros((len(neighboring_areas), max([len(x) for x in neighboring_areas])), dtype=np.int)

for i in range(len(neighboring_areas)):
    region_arr[i, :len(neighboring_areas[i])] = neighboring_areas[i]
    
np.savetxt('TwoTargetRegions.csv', region_arr[:2], delimiter=',')